In [1]:
import torch
import torchvision.datasets as dsets
from torchvision import transforms
import random


# check device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# set seed for reproducibility
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

torch.__version__, device

('1.7.0+cu101', 'cuda')

In [2]:
def train(model, criterion, optimizer, train_loader, n_epochs, device=device):
  print('>>> Learning started. It takes sometime.')

  total_batch = len(train_loader)
  for epoch in range(n_epochs):
    avg_cost = 0
    for x_train, y_train in train_loader:
      x_train = x_train.to(device)
      y_train = y_train.to(device)

      hypothesis = model(x_train)
      cost = criterion(hypothesis, y_train)

      optimizer.zero_grad()
      cost.backward()
      optimizer.step()

      avg_cost += cost/total_batch
    print(f'[Epoch: {epoch+1:3}] cost = {avg_cost:10.6f}')
  print('>>> Learning Finished!')


def test(model, test_data, device=device):
  with torch.no_grad():
    x_test = test_data.data.view(len(test_data), 1, 28, 28).float().to(device)
    y_test = test_data.targets.float().to(device)

    pred = model(x_test)
    correct_pred = pred.argmax(1) == y_test
    acc = correct_pred.float().mean()
    print(f'>>> Test Accuracy: {acc:10.6f}')

## Train CNN model with MNIST
- DNN과 마찬가지로 torch.nn.Module 상속해 모델 정의
- 각 CNN layer는 Conv2d + Relu + MaxPool2d로 구성
- 마지막 단계에서 view를 통한 flatten 작업 및 fc layer 통과

In [3]:
# set parameters
learning_rate = 0.001
n_epochs = 15
batch_size = 100

# prepare data
mnist_train = dsets.MNIST(root='mnist_data/', train=True, transform=transforms.ToTensor(), download=True)
mnist_test = dsets.MNIST(root='mnist_data/', train=False, transform=transforms.ToTensor(), download=True)

train_loader = torch.utils.data.DataLoader(dataset=mnist_train, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader  = torch.utils.data.DataLoader(dataset=mnist_test, batch_size=batch_size, shuffle=True, drop_last=True)

In [4]:
class CNN(torch.nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.layer1 = torch.nn.Sequential(
        torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(2),
    )
    self.layer2 = torch.nn.Sequential(
        torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(2),
    )
    self.fc = torch.nn.Linear(64*7*7, 10, bias=True)
    torch.nn.init.xavier_uniform_(self.fc.weight)

  def forward(self, x):
    output = self.layer1(x)
    output = self.layer2(output)

    output = output.view(output.size(0), -1)
    output = self.fc(output)
    return output

In [5]:
%%time
# generate model
model = CNN().to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# train model
train(model, criterion, optimizer, train_loader, n_epochs, device)

>>> Learning started. It takes sometime.
[Epoch:   1] cost =   0.220191
[Epoch:   2] cost =   0.060927
[Epoch:   3] cost =   0.046081
[Epoch:   4] cost =   0.036583
[Epoch:   5] cost =   0.029966
[Epoch:   6] cost =   0.026065
[Epoch:   7] cost =   0.020669
[Epoch:   8] cost =   0.018505
[Epoch:   9] cost =   0.015516
[Epoch:  10] cost =   0.012905
[Epoch:  11] cost =   0.011503
[Epoch:  12] cost =   0.008773
[Epoch:  13] cost =   0.008104
[Epoch:  14] cost =   0.005991
[Epoch:  15] cost =   0.006965
>>> Learning Finished!
CPU times: user 1min 20s, sys: 1.65 s, total: 1min 21s
Wall time: 1min 22s


In [6]:
# test model
test(model, mnist_test, device)

>>> Test Accuracy:   0.978300


In [7]:
# # clear memory
# import gc
# del model
# gc.collect()

## Larger Model
- 모델의 깊이를 키워 정확도 향상 시도
- 모델이 깊어진다고 해서 정확도가 항상 올라가는 것은 아님. 효율적인 모델 학습이 중요

In [8]:
class CNN2(torch.nn.Module):
  def __init__(self):
    super(CNN2, self).__init__()
    self.keep_prob = 0.5

    self.layer1 = torch.nn.Sequential(
        torch.nn.Conv2d(1, 32, 3, 1, padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.layer2 = torch.nn.Sequential(
        torch.nn.Conv2d(32, 64, 3, 1, padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.layer3 = torch.nn.Sequential(
        torch.nn.Conv2d(64, 128, 3, 1, padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
    )
    self.fc1 = torch.nn.Linear(128*4*4, 5**4, bias=True)
    torch.nn.init.xavier_uniform_(self.fc1.weight)
    self.layer4 = torch.nn.Sequential(
        self.fc1,
        torch.nn.ReLU(),
        torch.nn.Dropout(p=1-self.keep_prob)
    )
    self.fc2 = torch.nn.Linear(5**4, 10, bias=True)
    torch.nn.init.xavier_uniform_(self.fc2.weight)
  
  
  def forward(self, x):
    output = self.layer1(x)
    output = self.layer2(output)
    output = self.layer3(output)
    output = output.view(output.size(0), -1)
    output = self.layer4(output)
    output = self.fc2(output)
    return output

In [9]:
%%time
# generate model
model = CNN2().to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# train model
train(model, criterion, optimizer, train_loader, n_epochs, device)

>>> Learning started. It takes sometime.
[Epoch:   1] cost =   0.211705
[Epoch:   2] cost =   0.054164
[Epoch:   3] cost =   0.038617
[Epoch:   4] cost =   0.029678
[Epoch:   5] cost =   0.024067
[Epoch:   6] cost =   0.020131
[Epoch:   7] cost =   0.016899
[Epoch:   8] cost =   0.014031
[Epoch:   9] cost =   0.013088
[Epoch:  10] cost =   0.012663
[Epoch:  11] cost =   0.010881
[Epoch:  12] cost =   0.010647
[Epoch:  13] cost =   0.007446
[Epoch:  14] cost =   0.008707
[Epoch:  15] cost =   0.007447
>>> Learning Finished!
CPU times: user 1min 29s, sys: 773 ms, total: 1min 29s
Wall time: 1min 30s


In [10]:
# test model
test(model, mnist_test, device)

>>> Test Accuracy:   0.987600
